# Проект по SQL-запросам

Компания купила крупный сервис для чтения книг по подписке. Для исследования предоставлена база данных, которую необходимо проанализировать. В ней хранится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных



**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Оcновные задачи

- Посчитать сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — чтобы исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Устанавливаем параметры и сохраняем коннектор:

In [2]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Чтение и просмотр первых строк таблиц

In [3]:
query = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
display(books.head(5))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query_1 = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query_1, con = engine)
display(authors.head(5))

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query_2 = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query_2, con = engine)
display(publishers.head(5))

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query_3 = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query_3, con = engine)
display(ratings.head(5))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query_4 = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query_4, con = engine)
display(reviews.head(5))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ данных

Прежде чем приступить к анализу данных, создадим фукцию для удоства вывода результатов SQL-запросов:

In [8]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

### Количество книг, выпущенное после 1 января 2000 года

In [9]:
count_books = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''
select(count_books)

,count
0,821


Согласно полученным данным, после 1 января 2000 года всего выпущено книг в количестве 821.

### Подсчет для каждой книги количества обзоров и средней оценки

In [10]:
rating_and_review = '''
SELECT 
       books.title AS title,
       COUNT(DISTINCT reviews.text) AS cnt_reviews,
       AVG(ratings.rating) AS avg_rating
FROM books
FULL JOIN reviews ON books.book_id = reviews.book_id
FULL JOIN ratings ON reviews.book_id = ratings.book_id
WHERE books.title IS NOT NULL
GROUP BY books.title,
         books.book_id
ORDER BY cnt_reviews DESC
LIMIT 10
'''
select(rating_and_review)

,title,cnt_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,Eat Pray Love,6,3.395833
3,The Alchemist,6,3.789474
4,The Catcher in the Rye,6,3.825581
5,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,Outlander (Outlander #1),6,4.125000
9,The Book Thief,6,4.264151


Здесь представлены данные с названиями книг и количеством обзоров и средней оценкой для каждой книги. Наибольшее количество обзоров(7) получила книга "Twilight (Twilight #1)".

### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
big_publisher = '''
SELECT publishers.publisher,
       COUNT(books.book_id)
FROM publishers
FULL JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY COUNT(books.book_id) DESC
LIMIT 1
'''
select(big_publisher)

,publisher,count
0,Penguin Books,42


Согласно исследованию, издательством, которое выпустило наибольшее число книг, является "Penguin Books". Данное издательство выпустило 42 книги.

### Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками

In [12]:
best_author = '''
SELECT i.author,
       AVG(avg_rating) AS avg_rating_all_books
FROM 
(SELECT  
        authors.author AS author,
        AVG(ratings.rating) AS avg_rating
FROM authors
JOIN books ON authors.author_id = books.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY author,
         books.book_id
HAVING COUNT(ratings.rating_id) > 50
) AS i

GROUP BY i.author
ORDER BY AVG(avg_rating) DESC
LIMIT 1
'''
select(best_author)

,author,avg_rating_all_books
0,J.K. Rowling/Mary GrandPré,4.283844


J.K. Rowling/Mary GrandPré является  автором с самой высокой средней оценкой книг, при учете книг с 50 и более оценками.

### Рассчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
avg_cnt_reviews = '''
SELECT 
      AVG(m.cnt_text) avg_cnt_reviews
FROM
(SELECT  
        k.username,
        COUNT(text) AS cnt_text
FROM reviews
INNER JOIN (SELECT  
                  username,
                  COUNT(rating_id) AS cnt_rating
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50) AS k
ON k.username = reviews.username
GROUP BY k.username) AS m
'''
select(avg_cnt_reviews)

,avg_cnt_reviews
0,24.333333


24 - среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Вывод

Проанализировав данные и ответив на основные вопросы исследования, получены следующие результаты:
- после 1 января 2000 года всего выпущено книг в количестве 821
- книга "Twilight (Twilight #1)" получила 7 обзоров, что является наибольшим количеством среди других книг
- издательством, которое выпустило наибольшее число книг, является "Penguin Books". Данное издательство выпустило 42 книги
- J.K. Rowling/Mary GrandPré является  автором с самой высокой средней оценкой книг, при учете книг с 50 и более оценками
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24,33 обзора.